In [7]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt

In [8]:
class SimAnneal_arc(object):
    def __init__(self,coords,eta_0,alpha,delta,rho,M,T=-1, stopping_T=-1, stopping_iter=-1):
        '''tau can be specified to be any lower bound on the minimum at initialisation'''
        '''M is the number of iterations for which the same value of tau is recycled'''
        self.coords = coords
        self.eta_0 = eta_0
        self.alpha = alpha
        self.rho = rho
        self.tau = delta + self.eta_0
        self.M = M
        self.N = len(coords)
        self.T = math.sqrt(self.N)/math.log(2) if T == -1 else T
        #self.T = math.sqrt(self.N) /2 if T == -1 else T
        self.T_save = self.T  
        self.stopping_temperature = 1e-16 if stopping_T == -1 else stopping_T
        self.stopping_iter = 100000 if stopping_iter == -1 else stopping_iter
        self.iteration = 1
        self.acceptances = 0
        self.small_acceptances = 0
        self.worse_acceptances = 0
        self.nodes = [i for i in range(self.N)]

        self.best_solution = None
        self.best_fitness = float("Inf")
        self.fitness_list = []
        self.solution_history =[]

    def initial_solution(self):
        """
        Greedy algorithm to get an initial solution (closest-neighbour).
        """
        random.seed(2000)
        cur_node = random.choice(self.nodes)  # start from a random node
        solution = [cur_node]

        free_nodes = set(self.nodes)
        free_nodes.remove(cur_node)
        while free_nodes:
            next_node = min(free_nodes, key=lambda x: self.dist(cur_node, x))  # nearest neighbour
            free_nodes.remove(next_node)
            solution.append(next_node)
            cur_node = next_node

        cur_fit = self.fitness(solution)
        if cur_fit < self.best_fitness:  # If best found so far, update best fitness
            self.best_fitness = cur_fit
            self.best_solution = solution
        self.fitness_list.append(cur_fit)
        return solution, cur_fit

    def dist(self, node_0, node_1):
        """
        Euclidean distance between two nodes.
        """
        coord_0, coord_1 = self.coords[node_0], self.coords[node_1]
        return math.sqrt((coord_0[0] - coord_1[0]) ** 2 + (coord_0[1] - coord_1[1]) ** 2)

    def fitness(self, solution):
        """
        Total distance of the current solution path.
        """
        cur_fit = 0
        for i in range(self.N):
            cur_fit += self.dist(solution[i % self.N], solution[(i + 1) % self.N])
        return cur_fit

    def p_accept(self, candidate_fitness):
        """
        Probability of accepting if the candidate is worse than current.
        Depends on the current temperature and difference between candidate and current.
        """
        return math.exp(-abs(candidate_fitness - self.cur_fitness) / self.T)

    def improved_p_accept(self, candidate_fitness):
        """
        Probability of accepting if the candidate is worse than current.
        Depends on the current temperature and difference between candidate and current.
        """
        #print('current fitness is')
        #print(self.cur_fitness)
        return math.exp(-abs(self.alpha*candidate_fitness + np.arctan(candidate_fitness + self.tau)/self.T - (self.alpha*self.cur_fitness + np.arctan(self.cur_fitness + self.tau))/self.T))
    
    def accept(self, candidate):
        """
        Accept with probability 1 if candidate is better than current.
        Accept with probabilty p_accept(..) if candidate is worse.
        """
        candidate_fitness = self.fitness(candidate)
        if candidate_fitness < self.cur_fitness:
            if abs(((candidate_fitness - self.cur_fitness)/self.cur_fitness)*100) < 1:
                self.small_acceptances += 1 
            self.cur_fitness, self.cur_solution = candidate_fitness, candidate
            self.acceptances += 1
            if candidate_fitness < self.best_fitness:
                self.best_fitness, self.best_solution = candidate_fitness, candidate
        else:
            #print(self.p_accept(candidate_fitness))
            if random.random() < self.p_accept(candidate_fitness):
                if abs(((candidate_fitness - self.cur_fitness)/self.cur_fitness)*100) < 1:
                    self.small_acceptances += 1 
                self.cur_fitness, self.cur_solution = candidate_fitness, candidate
                self.acceptances += 1
                self.worse_acceptances += 1

    def improved_accept(self, candidate):
        """
        Accept with probability 1 if candidate is better than current.
        Accept with probabilty p_accept(..) if candidate is worse.
        """
        candidate_fitness = self.fitness(candidate)
        if candidate_fitness < self.cur_fitness:
            if abs(((candidate_fitness - self.cur_fitness)/self.cur_fitness)*100) < 1:
                self.small_acceptances += 1 
            self.cur_fitness, self.cur_solution = candidate_fitness, candidate
            self.acceptances += 1
            if candidate_fitness < self.best_fitness:
                self.best_fitness, self.best_solution = candidate_fitness, candidate
        else:
            #print(self.improved_p_accept(candidate_fitness))
            if random.random() < self.improved_p_accept(candidate_fitness):
                if abs(((candidate_fitness - self.cur_fitness)/self.cur_fitness)*100) < 1:
                    self.small_acceptances += 1 
                self.cur_fitness, self.cur_solution = candidate_fitness, candidate
                self.acceptances += 1
                self.worse_acceptances += 1

    def anneal(self):
        """
        Execute simulated annealing algorithm.
        """
        # Initialize with the greedy solution.
        #print(self.coords)
        self.cur_solution, self.cur_fitness = self.initial_solution()
        self.solution_history.append(self.cur_solution)

        #print("Starting annealing.")
        while self.T >= self.stopping_temperature and self.iteration < self.stopping_iter:
            candidate = list(self.cur_solution)
            l = random.randint(2, self.N - 1)
            i = random.randint(0, self.N - l)
            candidate[i : (i + l)] = reversed(candidate[i : (i + l)])
            self.accept(candidate)
            self.iteration += 1
            self.T = math.sqrt(self.N)/math.log(self.iteration+1)
            self.fitness_list.append(self.cur_fitness)
            self.solution_history.append(self.cur_solution)

        #print("Best fitness obtained: ", self.best_fitness)
        improvement = 100 * (self.fitness_list[0] - self.best_fitness) / (self.fitness_list[0])
        #print(f"Improvement over greedy heuristic: {improvement : .2f}%")

    def arctan_anneal(self):
        """
        Execute improved simulated annealing algorithm.
        """
        # Initialize with the greedy solution.
        self.cur_solution, self.cur_fitness = self.initial_solution()
        self.solution_history.append(self.cur_solution)

        #print("Starting improved annealing.")
        while self.T >= self.stopping_temperature and self.iteration < self.stopping_iter:
            # update rule for tau
            #print('tau is: ',self.tau)
            if self.iteration % self.M == 0 and self.iteration > 1:
                self.tau = self.tau - 1/(1 + self.rho)*(self.best_fitness + self.tau) + self.eta_0
            candidate = list(self.cur_solution)
            l = random.randint(2, self.N - 1)
            i = random.randint(0, self.N - l)
            candidate[i : (i + l)] = reversed(candidate[i : (i + l)])
            self.improved_accept(candidate)
            self.iteration += 1
            self.T = math.sqrt(self.N)/math.log(self.iteration+1)
            self.solution_history.append(self.cur_solution)
            self.fitness_list.append(self.cur_fitness)

        improvement = 100 * (self.fitness_list[0] - self.best_fitness) / (self.fitness_list[0])

    def visualize_routes(self):
        """
        Visualize the TSP route with matplotlib.
        """
        visualize_tsp.plotTSP([self.best_solution], self.coords)

    def plot_learning(self):
        """
        Plot the fitness through iterations.
        """
        plt.plot([i for i in range(len(self.fitness_list))], self.fitness_list)
        plt.ylabel("Fitness")
        plt.xlabel("Iteration")
        plt.show()

    def animateSolutions(self,coords):
        animated_visualizer.animateTSP(self.solution_history, coords)